<a href="https://colab.research.google.com/github/jasmeen2105/Loan_Approval_System/blob/main/Loan_Approval_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas scikit-learn joblib imbalanced-learn

In [ ]:
import numpy as np
import pandas as pd
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE  # For handling imbalanced data

In [ ]:
import zipfile

# Unzip the file
with zipfile.ZipFile('/content/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/archive') # Extract to a new directory

# Update data_path to the extracted directory
data_path = '/content/archive'

In [ ]:
import pandas as pd # Make sure pandas is imported

df = pd.read_csv('/content/archive/loan_data.csv')

In [ ]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [ ]:
df.shape

(45000, 14)

In [ ]:
df.describe()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,45000.000000,4.500000e+04,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,27.764178,8.031905e+04,5.410333,9583.157556,11.006606,0.139725,5.867489,632.608756,0.222222
std,6.045108,8.042250e+04,6.063532,6314.886691,2.978808,0.087212,3.879702,50.435865,0.415744
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,390.000000,0.000000
25%,24.000000,4.720400e+04,1.000000,5000.000000,8.590000,0.070000,3.000000,601.000000,0.000000
50%,26.000000,6.704800e+04,4.000000,8000.000000,11.010000,0.120000,4.000000,640.000000,0.000000
75%,30.000000,9.578925e+04,8.000000,12237.250000,12.990000,0.190000,8.000000,670.000000,0.000000
max,144.000000,7.200766e+06,125.000000,35000.000000,20.000000,0.660000,30.000000,850.000000,1.000000


In [ ]:
# Handle Missing Values (Fill NaN values with the mean for numerical columns)
df.fillna(df.mean(numeric_only=True), inplace=True)

In [ ]:
# Encode Categorical Data (Convert strings to numbers)
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder

label_encoders = {}  # Store encoders for future use
for col in df.select_dtypes(include=['object']).columns:
    encoder = LabelEncoder()
    df[col] = encoder.fit_transform(df[col])
    label_encoders[col] = encoder  # Save encoder for later use

In [ ]:
X = df.drop(columns=['loan_status'])
X.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file
0,22.0,0,4,71948.0,0,3,35000.0,4,16.02,0.49,3.0,561,0
1,21.0,0,3,12282.0,0,2,1000.0,1,11.14,0.08,2.0,504,1
2,25.0,0,3,12438.0,3,0,5500.0,3,12.87,0.44,3.0,635,0
3,23.0,0,1,79753.0,0,3,35000.0,3,15.23,0.44,2.0,675,0
4,24.0,1,4,66135.0,1,3,35000.0,3,14.27,0.53,4.0,586,0


In [ ]:
y = df['loan_status']
y.head()

,loan_status
0,1
1,0
2,1
3,1
4,1


In [ ]:
# Define Features (X) and Target (y)
X = df.drop(columns=['loan_status'])  # Features (exclude target column)
y = df['loan_status']  # Target (0 = Rejected, 1 = Approved)

In [ ]:
# Handle Class Imbalance Using SMOTE (Optional)
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split # Import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42) # Split the resampled data

In [ ]:
# Normalize Features (Decision Tree doesn’t require scaling, but helps)
from sklearn.preprocessing import StandardScaler # Import StandardScaler here
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train the Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier # Import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(
    max_depth=10,         # Limits tree depth (prevents overfitting)
    min_samples_split=10,  # Minimum samples needed to split
    min_samples_leaf=5,    # Minimum samples in a leaf node
    random_state=42
)
dt_model.fit(X_train_scaled, y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=5, min_samples_split=10,
                       random_state=42)

In [ ]:
# Save the trained model & scaler
joblib.dump(dt_model, "dt_loan_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print(" Decision Tree Model trained and saved successfully!")

 Decision Tree Model trained and saved successfully!


In [ ]:
# Load the Trained Model & Scaler
dt_model = joblib.load("dt_loan_model.pkl")
scaler = joblib.load("scaler.pkl")

In [ ]:
# Evaluate Model Performance
y_pred = dt_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f" Model Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

 Model Accuracy: 90.50%

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.89      0.90      7023
           1       0.89      0.92      0.91      6977

    accuracy                           0.91     14000
   macro avg       0.91      0.91      0.90     14000
weighted avg       0.91      0.91      0.90     14000



In [ ]:
import numpy as np
import joblib

# Load the Trained Model & Scaler
dt_model = joblib.load("dt_loan_model.pkl")
scaler = joblib.load("scaler.pkl")


In [ ]:
# Accept User Input
print("\n Enter the Applicant Details for Loan Prediction:")

gender = int(input("Gender (Male=1, Female=0): "))
age = int(input("Age: "))
married = int(input("Married (Yes=1, No=0): "))
dependents = int(input("Number of Dependents: "))
education = int(input("Education (Graduate=1, Not Graduate=0): "))
self_employed = int(input("Self Employed (Yes=1, No=0): "))
credit_history = int(input("Credit History (Good=1, Bad=0): "))
property_area = int(input("Property Area (Urban=2, Semiurban=1, Rural=0): "))
applicant_income = float(input("Applicant Income: "))
coapplicant_income = float(input("Co-applicant Income: "))
loan_amount = float(input("Loan Amount: "))
loan_term = int(input("Loan Term (in months): "))
existing_loan = int(input("Existing Loan (Yes=1, No=0): "))


 Enter the Applicant Details for Loan Prediction:
Gender (Male=1, Female=0): 1
Age: 34
Married (Yes=1, No=0): 1
Number of Dependents: 2
Education (Graduate=1, Not Graduate=0): 1
Self Employed (Yes=1, No=0): 1
Credit History (Good=1, Bad=0): 1
Property Area (Urban=2, Semiurban=1, Rural=0): 2
Applicant Income: 60000
Co-applicant Income: 45000
Loan Amount: 100000
Loan Term (in months): 6
Existing Loan (Yes=1, No=0): 0


In [ ]:
# Convert Input into NumPy Array
new_applicant = np.array([[gender, age, married, dependents, education, self_employed,
                            credit_history, property_area, applicant_income, coapplicant_income,
                            loan_amount, loan_term, existing_loan]])


In [ ]:
# Scale Features (Apply same transformation as training data)
new_applicant_scaled = scaler.transform(new_applicant)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Make Prediction
prediction = dt_model.predict(new_applicant_scaled)
prediction_probabilities = dt_model.predict_proba(new_applicant_scaled)

In [ ]:
# Display Loan Approval Status
status = "Approved" if prediction[0] == 1 else "Rejected"
print(f"\n🔹 Loan Prediction Result: {status}")


🔹 Loan Prediction Result: Approved


In [ ]:
# Show Probability Score
print("\n🔹 Prediction Probabilities (Rejection vs Approval):", prediction_probabilities)


🔹 Prediction Probabilities (Rejection vs Approval): [[0. 1.]]
